In [2]:
%pip install pyspark

  Using cached pyspark-4.1.1-py2.py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pyspark]m1/2 [pyspark]
Note: you may need to restart the kernel to use updated packages.


### Importações do PySpark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

### Spark Session
Spark Session initializer

In [ ]:
spark = SparkSession.builder. \
    appName("SilverOrders"). \
    getOrCreate()

### Bronze
Bronze layer reading

In [ ]:
order_items = spark.read.csv("data/bronze/olist_order_items_dataset.csv", header=True, inferSchema=True)
orders = spark.read.csv("data/bronze/olist_orders_dataset.csv", header=True, inferSchema=True)
payments = spark.read.csv("data/bronze/olist_order_payments_dataset.csv", header=True, inferSchema=True)

### Silver
Silver transformation

In [ ]:
payments_agg = payments.groupBy("order_id")\
  .agg(sum("payment_value").alias("total_payment_value"))

silver_orders = orders.join(order_items, "order_id", "inner")\
  .join(payments_agg, "order_id", "left")

silver_orders.write \
  .format("jdbc") \
  .option("url", "jdbc:postgresql://realtime_postgres:5432/realtime_dw") \
  .option("dbtable", "silver_orders") \
  .option("user", "realtime") \
  .option("password", "realtime123") \
  .mode("overwrite") \
  .save()

In [ ]:
spark.stop()